In [ ]:
#本文将复现残差网络，尽量一模块化得形式复现网络
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Input,GlobalAveragePooling2D,Dense
from tensorflow.keras.activations import relu

In [ ]:
#残差网络中主要是残差块，那么残差块分两种，如图所示残差边含卷积得是因为维度会发生改变
#不含残差边得维度不改变
#开始定义block
class Block(Model):
    def __init__(self,input_channels,output_channels,use_conv=False,identity_strides=1):
        super().__init__()
        self.conv1=Conv2D(input_channels,kernel_size=1,strides=identity_strides)
        self.conv2=Conv2D(input_channels,kernel_size=3,strides=1,padding='same')
        self.conv3=Conv2D(output_channels,kernel_size=1,strides=1,padding='same')
        self.conv_identity=None
        if use_conv:
            self.conv_identity=Conv2D(output_channels,kernel_size=1,strides=identity_strides,padding='same')
            
        self.bn1=BatchNormalization()
        self.bn2=BatchNormalization()
        self.bn3=BatchNormalization()
        self.bn_identity =BatchNormalization()
        
    def call(self,X):
        #这里采用传统残差
        Y=self.conv1(X)
        Y = self.bn1(Y)
        Y= relu(Y)
        
        Y=self.conv2(Y)
        Y = self.bn2(Y)
        Y= relu(Y)
        
        Y=self.conv3(Y)
        Y = self.bn3(Y)
        Y= relu(Y)
        
        if self.conv_identity is not None:
            X=self.conv_identity(X)
            X=self.bn_identity(X)
        
        Y+=X
        Y=relu(Y)
        return Y
        

In [ ]:
# #测试残差快是否运算正确
# block =Block(64,256,False,1)
# X = tf.random.uniform((4, 56, 56,256))
# Y = block(X)
# Y.shape


In [ ]:
# block =Block(128,512,True,2)
# X = tf.random.uniform((4, 56, 56,256))
# Y = block(X)
# Y.shape

In [ ]:
base_layer = tf.keras.models.Sequential([
    Conv2D(input_shape=(224,224,3),filters=64,kernel_size=7,strides=2,padding='same'),
    BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    MaxPooling2D(pool_size=3, strides=2, padding='same')
    
])
base_layer.summary()

In [ ]:
#接下来 我们要去组件我们的ResNet 50 了
from tensorflow.python.keras import backend
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.utils import layer_utils
from tensorflow.keras import optimizers, losses, initializers
from tensorflow.keras.activations import relu
def ResNet50(input_shape=(224, 224, 3), input_tensor=None,classes=1000):
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
            
    X=Conv2D(filters=64,kernel_size=7,strides=2,padding='same')(img_input)
    X= BatchNormalization()(X)
    X=relu(X)
    X= MaxPooling2D(pool_size=3, strides=2, padding='same')(X)
    #stage1
    X=Block(64,256,True,1)(X)
    X=Block(64,256,False)(X)
    X=Block(64,256,False)(X)
    
    #Stage2
    X=Block(128,512,True,2)(X)
    X=Block(128,512,False)(X)
    X=Block(128,512,False)(X)
    
    #Stage3
    X=Block(256,1024,True,2)(X)
    X=Block(256,1024,False)(X)
    X=Block(256,1024,False)(X)
    
    X=Block(512,2048,True,2)(X)
    X=Block(512,2048,False)(X)
    X=Block(512,2048,False)(X)
    #这里的输出有点大，我们不用Flatten了 ，采用GlobalAveragePooling2D()吧
    #X = Flatten()(X)
    #这里我才用全局平均池化来做这件事情
    X=GlobalAveragePooling2D()(X)
    X=Dense(classes,activation='softmax')(X)

    if input_tensor is not None:
        inputs = layer_utils.get_source_inputs(input_tensor)
    else:
         inputs = img_input
    model = training.Model(inputs, X, name='ResNet50')
    return model

In [ ]:
model =ResNet50(input_shape=(224,224,3),classes=270)
model.summary()

In [ ]:

#数据生成器
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_dir='../input/100-bird-species/train'
val_data_dir='../input/100-bird-species/valid'
test_data_dir='../input/100-bird-species/test'
train_datagen=ImageDataGenerator(
 rescale=1/255.0, #归一化
 rotation_range=10, #图片随机旋转的角度
 zoom_range=0.05, # 图片随机缩放的幅度
 width_shift_range=0.05, #图片水平偏移的幅度
 height_shift_range=0.05, #图片垂直偏移的幅度
 shear_range=0.05,#裁剪强度
    horizontal_flip=True,#水平翻转
 fill_mode='nearest')#；‘constant’，‘nearest’，‘reflect’或‘wrap’之一，当进行变换时超出边界的点将根据本参数给定的方法进行处理) #验证集的占比

batch_size =50 #
#训练集
train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir, #图片文件路径
    target_size=(224,224),#读入时设置的大小
    color_mode='rgb',#颜色模式
    batch_size=batch_size,
    class_mode='categorical', #分类模式
    shuffle=True, #乱序
    seed=42 #随机数粽子
)
#验证集
val_datagen=ImageDataGenerator(rescale=1/255.0)
valid_generator = val_datagen.flow_from_directory(
    directory=val_data_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

#测试集
test_datagen = ImageDataGenerator(rescale=1/255.0) #测试机只设置归一化
test_generator = test_datagen.flow_from_directory(
  directory=test_data_dir,
  target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode=None,
    shuffle=False,
    seed=42
)

In [ ]:
from tensorflow.keras.optimizers import Adam
#模型配置并训练
lr=0.0001
epochs=15
opt=Adam(lr=lr,decay=lr/(epochs/0.5))
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc'])
model.fit(train_generator,validation_data=valid_generator,
          steps_per_epoch=train_generator.n//train_generator.batch_size,
          validation_steps=valid_generator.n//valid_generator.batch_size,
          epochs=epochs
         )

In [ ]:
score = model.evaluate_generator(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
score = model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
